In [1]:
using JSON

#load OD pair-route incidence
odPairRoute = readall("od_pair_route_incidence_Sioux.json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readall("link_route_incidence_Sioux.json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readall("od_pair_label_dict_Sioux_refined.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__Sioux_refined.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readall("link_label_dict_Sioux.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readall("link_label_dict_Sioux_.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readall("node_link_incidence_Sioux.json");
nodeLink = JSON.parse(nodeLink);

In [2]:
include("load_network_uni-class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
ta_data = load_ta_network("Sioux Falls");

In [4]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [5]:
############
#Read in the demand file
file = open("../data_original/SiouxFalls_trips.txt")
demands = Dict{(Int64,Int64), Float64}()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = int(split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = int(pair_vals[1]), float(pair_vals[2])
                demands[(s,t)] = demand 
            end
        end
    end
end                
close(file)

In [6]:
demands

Dict{(Int64,Int64),Float64} with 576 entries:
  (23,8)  => 300.0
  (8,23)  => 300.0
  (6,9)   => 400.0
  (18,3)  => 0.0
  (21,13) => 600.0
  (22,16) => 1200.0
  (15,17) => 1500.0
  (22,14) => 1200.0
  (13,9)  => 600.0
  (23,10) => 1800.0
  (24,22) => 1100.0
  (8,9)   => 800.0
  (9,1)   => 500.0
  (17,14) => 700.0
  (16,6)  => 900.0
  (16,12) => 700.0
  (16,13) => 600.0
  (22,2)  => 100.0
  (24,3)  => 0.0
  (7,16)  => 1400.0
  (1,12)  => 200.0
  (4,5)   => 500.0
  (11,8)  => 800.0
  (18,21) => 100.0
  (19,4)  => 200.0
  ⋮       => ⋮

In [7]:
odPairLabel_

Dict{String,Any} with 552 entries:
  "306" => {14,7}
  "153" => {7,16}
  "1"   => {1,2}
  "54"  => {3,9}
  "519" => {23,13}
  "101" => {5,10}
  "371" => {17,3}
  "41"  => {2,19}
  "464" => {21,4}
  "65"  => {3,20}
  "475" => {21,15}
  "447" => {20,10}
  "335" => {15,13}
  "362" => {16,18}
  "505" => {22,23}
  "491" => {22,8}
  "326" => {15,4}
  "543" => {24,14}
  "299" => {13,24}
  "168" => {8,7}
  "403" => {18,12}
  "159" => {7,22}
  "228" => {10,22}
  "332" => {15,10}
  "190" => {9,6}
  ⋮     => ⋮

In [8]:
demandsVec = zeros(552)

for i = 1:length(demandsVec)
    demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
end

In [9]:
demandsVec

552-element Array{Float64,1}:
  100.0
  100.0
  500.0
  200.0
  300.0
  500.0
  800.0
  500.0
 1300.0
  500.0
  200.0
  500.0
  300.0
    ⋮  
  500.0
  700.0
  400.0
  400.0
  300.0
  300.0
    0.0
  100.0
  400.0
  500.0
 1100.0
  700.0

In [10]:
for key=keys(odPairRoute)
    if contains(key, "552-")
        println(key)
    end
end

552-97335
552-97243
552-97246
552-97428
552-97365
552-97404
552-97376
552-97407
552-97247
552-97260
552-97389
552-97427
552-97366
552-97390
552-97297
552-97303
552-97381
552-97351
552-97370
552-97344
552-97327
552-97357
552-97286
552-97292
552-97361
552-97263
552-97434
552-97341
552-97290
552-97415
552-97317
552-97406
552-97319
552-97278
552-97285
552-97439
552-97378
552-97324
552-97288
552-97354
552-97379
552-97377
552-97405
552-97333
552-97259
552-97385
552-97320
552-97266
552-97373
552-97273
552-97438
552-97284
552-97244
552-97248
552-97343
552-97410
552-97355
552-97408
552-97306
552-97382
552-97371
552-97433
552-97316
552-97304
552-97262
552-97419
552-97302
552-97393
552-97422
552-97426
552-97239
552-97258
552-97256
552-97295
552-97305
552-97257
552-97289
552-97296
552-97353
552-97328
552-97318
552-97238
552-97299
552-97356
552-97383
552-97400
552-97245
552-97254
552-97298
552-97346
552-97279
552-97313
552-97414
552-97310
552-97374
552-97395
552-97309
552-97367
552-97307
552-97420


In [11]:
using JuMP

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 76
numRoute = 97442
numOD = 552

@defVar(m, linkFlow[1:numLinks])
@defVar(m, pathFlow[1:numRoute])

pathFlowSum = Dict()

for i=1:numOD
    pathFlowSum[i] = 0
    for j=1:numRoute
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()

for a=1:numLinks
    pathFlowLinkSum[a] = 0
    for j=1:numRoute
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowLinkSum[a] == linkFlow[a])
end

for j=1:numRoute
    @addConstraint(m, pathFlow[j] >= 0)
end

@defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   822292
Number of nonzeros in inequality constraint Jacobian.:    97442
Number of nonzeros in Lagrangian Hessian.............:       76

Total number of variables............................:    97518
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [12]:
getValue(linkFlow)

76-element Array{Float64,1}:
 26858.8 
  9114.95
 41567.5 
  5985.05
 36966.0 
 14590.4 
 23140.3 
 12228.2 
 18762.2 
 10872.2 
 16857.3 
  8449.11
 18964.3 
     ⋮   
  7522.87
  9648.14
 17742.9 
  6900.0 
  7518.63
 10082.1 
  8197.49
 10188.2 
  9851.86
 12400.0 
  9197.42
  8742.9 

In [13]:
getObjectiveValue(m)

5.567477778182257e6

In [14]:
outfile = open("flows_converge.txt", "w")

write(outfile, join(("From", "to", "Volume_Capacity"), "        "), "\n")

for i = 1:length(ta_data.start_node)
     n1, n2, n3 = ta_data.start_node[i], ta_data.end_node[i], getValue(linkFlow)[i]
     write(outfile, join((n1, n2, n3), "        "), "\n")
end

close(outfile)

In [15]:
flows = Dict{(Int64,Int64),Float64}()

for i = 1:length(ta_data.start_node)
    key = (ta_data.start_node[i], ta_data.end_node[i])
    flows[key] = getValue(linkFlow)[i]
end

flows

Dict{(Int64,Int64),Float64} with 76 entries:
  (15,19) => 20077.251086334516
  (20,21) => 6429.51139505354
  (22,20) => 6899.99991662029
  (23,22) => 10188.166207514116
  (10,11) => 20158.437834024782
  (8,9)   => 7842.684134268605
  (13,12) => 13999.999992200188
  (24,21) => 9197.421556040972
  (3,1)   => 36966.029178144025
  (19,17) => 13043.837949797946
  (4,5)   => 18762.17749560892
  (17,16) => 11652.934789231924
  (20,22) => 6899.999916595283
  (22,21) => 7518.630741265086
  (18,16) => 7123.989844491068
  (6,5)   => 9257.315790512012
  (16,17) => 8656.161902604787
  (22,15) => 17742.9101893503
  (1,2)   => 26858.82563702409
  (22,23) => 10082.07915153072
  (19,20) => 8862.150834280432
  (9,8)   => 7473.951523796909
  (11,12) => 9299.999938811083
  (3,4)   => 14590.39279754571
  (8,16)  => 12723.98975956997
  ⋮       => ⋮

In [16]:
using JuMP

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 76  # number of links
numNodes = 24  # number of nodes

@defVar(m, linkFlow[1:numLinks])

@defVar(m, odLinkFlow[keys(demands), 1:numLinks])
for (s,t) = keys(demands)
    for k = 1:numLinks
        @addConstraint(m, odLinkFlow[(s,t), k] >= 0)
    end
end

for k = 1:numLinks
    linkFlowPartial = sum([odLinkFlow[(s,t), k] for (s,t) = keys(demands)])
    @addConstraint(m, linkFlowPartial == linkFlow[k])
end

for l = 1:numNodes
    for (s,t) = keys(demands)
        if s != t
            odLinkFlowPartial = sum([nodeLink["$(l-1)-$(k-1)"] * odLinkFlow[(s,t), k] for k = 1:numLinks])
            if (l == s)
                @addConstraint(m, odLinkFlowPartial + demands[(s,t)] == 0)
            elseif (l == t)
                @addConstraint(m, odLinkFlowPartial - demands[(s,t)] == 0)
            else
                @addConstraint(m, odLinkFlowPartial == 0)
            end
        end
    end
end

@defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:76})

@setNLObjective(m, Min, f)

solve(m)

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   127756
Number of nonzeros in inequality constraint Jacobian.:    43776
Number of nonzeros in Lagrangian Hessian.............:       76

Total number of variables............................:    43852
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:    13324
Total number of inequality constraints...............:    43776
        inequality constraints with only lower bounds:    43776
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

:Optimal

In [17]:
getValue(linkFlow)

76-element Array{Float64,1}:
  4519.08
  8094.66
  4494.66
  5991.76
  8119.08
 14030.6 
  9973.71
 14006.4 
 18030.6 
  5300.0 
 18006.4 
  8806.5 
 15796.7 
     ⋮   
  8607.39
 10259.5 
 18409.9 
  7000.0 
  8619.54
  9626.21
  8400.44
  9661.82
  7861.83
 11121.4 
 10309.4 
  7902.98

In [18]:
getObjectiveValue(m)

4.231335283350912e6